In [16]:
# load packages for analysis
import os
import ast
import contextily as ctx
import dask.dataframe as dd
import dask_geopandas as dg
import fiona
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np
import networkx as nx
import osmnx as ox
import pandas as pd
import pyogrio
import seaborn as sns
from xml import etree
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler
from shapely.geometry import Point
from shapely.geometry import LineString, MultiLineString
import shapely.wkt
import statsmodels.api as sm

In [17]:
burbs = gpd.read_file("data/ODP/ct_suburbs.gpkg")

In [18]:
pp_elec =  pd.read_csv("Prepaid_electricity_by_suburb.csv")

In [19]:
pp_elec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7685 entries, 0 to 7684
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   area            7678 non-null   object 
 1   units_per_area  7685 non-null   float64
 2   txmonth         7685 non-null   object 
 3   number_in_area  7685 non-null   int64  
 4   tariff          7685 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 300.3+ KB


In [20]:
burbs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    778 non-null    int64   
 1   OFC_SBRB_N  778 non-null    object  
 2   Shape_Leng  778 non-null    float64 
 3   Shape_Area  778 non-null    float64 
 4   geometry    778 non-null    geometry
dtypes: float64(2), geometry(1), int64(1), object(1)
memory usage: 30.5+ KB


In [28]:
from geopy.geocoders import Nominatim
import time

# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# Function to get latitude and longitude from Nominatim geocoding
def get_coordinates(area_name):
    try:
        location = geolocator.geocode(area_name + ", Cape Town, South Africa")  # Include additional context for accuracy
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {area_name}: {e}")
        return None, None

# Apply the function to the 'area' column in pp_elec
pp_elec['latitude'], pp_elec['longitude'] = zip(*pp_elec['area'].apply(get_coordinates))

# Introduce a delay to avoid hitting rate limits
time.sleep(1)

# Display the result
import ace_tools as tools; tools.display_dataframe_to_user(name="pp_elec with Coordinates", dataframe=pp_elec)

Error geocoding BEACON VALLEY: Non-successful status code 403
Error geocoding EASTRIDGE: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=EASTRIDGE%2C+Cape+Town%2C+South+Africa&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error geocoding nan: unsupported operand type(s) for +: 'float' and 'str'
Error geocoding LENTEGEUR: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=LENTEGEUR%2C+Cape+Town%2C+South+Africa&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Error geocoding ROCKLANDS: Non-successful status code 503
Error geocoding TAFELSIG: Non-successful status code 503
Error geocoding TAFELSIG: Non-successful status code 503
Error geocoding BONTEHEUWEL: Non-success

ModuleNotFoundError: No module named 'ace_tools'